# Kafka Producer for Climate Data Streaming

The answers have been modified for portfolio purpose

Author: Grace Nathania

In [1]:
# loading necessary library
import csv
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime as dt
from pymongo import MongoClient

def find_latest_date():
    client = MongoClient()
    db = client.stopfire_db
    collection = db.data
    
    cursor = collection.find({},{'date': 1, '_id':0})
    
    date_list = []
    
    for doc in cursor:
        date = dt.datetime.strptime(doc['date'],'%d/%m/%y').date()
        date_list.append(date)
    
    return sorted(date_list)[-1]
    
    
def process_data(file_path, to_int, to_float):
    reader = csv.DictReader(open(file_path,'r'))
    result = list(reader)
    
    for data in result:
        
        for col in to_int:
            data[col] = int(data[col])
            
        for col in to_float:
            # Splitting precipitation value for easier query
            if col == 'precipitation ':
                item = data[col].strip()
                item_sz = len(item)
                flag = item[-1]
                value = float(item[:item_sz-1])
                data['precipitation_flag'] = flag
                data['precipitation_value'] = value
                data.pop('precipitation ')
            else:
                data[col] = float(data[col])
                
        data['producer'] = 1
        
    return result
    
def publish_message(producer_instance, topic_name, data):
    try:
        value_bytes = bytes(data, encoding='utf-8')
        key_bytes = bytes("1", encoding='utf-8')
        producer_instance.send(topic_name, value=value_bytes,key=key_bytes)
        print('Message published successfully. ' + data)
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
     
    
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['127.0.0.1:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':

    # Load and process climate_streaming.csv data
    col_int = ['GHI_w/m2','air_temperature_celcius']
    col_float = ['latitude','longitude','max_wind_speed','windspeed_knots','relative_humidity','precipitation ']

    data_list = process_data("Data/climate_streaming.csv", col_int, col_float)
    
    # Kafka producer
    topic = 'StopFire'
    print('Publishing records..')
    producer01 = connect_kafka_producer()
    
    latest_date = find_latest_date()
    current_date = latest_date + dt.timedelta(days=1) 
    
    while True:
        #print("\nseconds:", seconds,"\n")
        idx = random.randrange(0,len(data_list))
        
        to_send = data_list[idx]
        
        # generate current date
        date_time = dt.datetime.now().strftime("%d/%m/%y %H:%M:%S")
        to_send['created_date'] = date_time
        to_send['date'] = current_date.strftime('%d/%m/%y')
        
        
        # set dictionary data to string for sending
        to_send = str(to_send)
        
        publish_message(producer01, topic, to_send)
        current_date = current_date + dt.timedelta(days=1)
            
        # sending every 10 seconds
        sleep(10)

Publishing records..
Message published successfully. {'max_wind_speed': 9.9, 'precipitation_value': 0.04, 'date': '01/01/19', 'GHI_w/m2': 66, 'air_temperature_celcius': 7, 'windspeed_knots': 5.7, 'latitude': -36.429, 'producer': 1, 'precipitation_flag': 'G', 'created_date': '25/06/21 23:17:42', 'relative_humidity': 37.9, 'longitude': 141.5759}
Message published successfully. {'max_wind_speed': 11.1, 'precipitation_value': 0.0, 'date': '02/01/19', 'GHI_w/m2': 109, 'air_temperature_celcius': 12, 'windspeed_knots': 7.0, 'latitude': -37.9072, 'producer': 1, 'precipitation_flag': 'G', 'created_date': '25/06/21 23:17:52', 'relative_humidity': 43.3, 'longitude': 141.9183}
Message published successfully. {'max_wind_speed': 12.0, 'precipitation_value': 0.0, 'date': '03/01/19', 'GHI_w/m2': 119, 'air_temperature_celcius': 14, 'windspeed_knots': 9.1, 'latitude': -37.856, 'producer': 1, 'precipitation_flag': 'G', 'created_date': '25/06/21 23:18:02', 'relative_humidity': 51.0, 'longitude': 143.416}


Message published successfully. {'max_wind_speed': 15.0, 'precipitation_value': 0.0, 'date': '27/01/19', 'GHI_w/m2': 122, 'air_temperature_celcius': 14, 'windspeed_knots': 9.1, 'latitude': -37.4661, 'producer': 1, 'precipitation_flag': 'G', 'created_date': '25/06/21 23:22:03', 'relative_humidity': 48.2, 'longitude': 143.2419}
Message published successfully. {'max_wind_speed': 16.9, 'precipitation_value': 0.08, 'date': '28/01/19', 'GHI_w/m2': 66, 'air_temperature_celcius': 7, 'windspeed_knots': 6.2, 'latitude': -36.3328, 'producer': 1, 'precipitation_flag': 'G', 'created_date': '25/06/21 23:22:13', 'relative_humidity': 37.2, 'longitude': 146.0355}
Message published successfully. {'max_wind_speed': 12.0, 'precipitation_value': 0.02, 'date': '29/01/19', 'GHI_w/m2': 99, 'air_temperature_celcius': 11, 'windspeed_knots': 6.5, 'latitude': -37.365, 'producer': 1, 'precipitation_flag': 'G', 'created_date': '25/06/21 23:22:23', 'relative_humidity': 43.8, 'longitude': 148.052}
Message published s

Message published successfully. {'max_wind_speed': 8.9, 'precipitation_value': 0.0, 'date': '22/02/19', 'GHI_w/m2': 103, 'air_temperature_celcius': 12, 'windspeed_knots': 4.4, 'latitude': -37.466, 'producer': 1, 'precipitation_flag': 'I', 'created_date': '25/06/21 23:26:23', 'relative_humidity': 49.4, 'longitude': 148.1}
Message published successfully. {'max_wind_speed': 8.0, 'precipitation_value': 0.0, 'date': '23/02/19', 'GHI_w/m2': 72, 'air_temperature_celcius': 8, 'windspeed_knots': 3.9, 'latitude': -37.386, 'producer': 1, 'precipitation_flag': 'I', 'created_date': '25/06/21 23:26:33', 'relative_humidity': 43.3, 'longitude': 148.043}
Message published successfully. {'max_wind_speed': 19.0, 'precipitation_value': 0.35, 'date': '24/02/19', 'GHI_w/m2': 122, 'air_temperature_celcius': 14, 'windspeed_knots': 13.7, 'latitude': -37.236, 'producer': 1, 'precipitation_flag': 'G', 'created_date': '25/06/21 23:26:43', 'relative_humidity': 48.1, 'longitude': 141.176}
Message published successf

Message published successfully. {'max_wind_speed': 8.0, 'precipitation_value': 0.0, 'date': '20/03/19', 'GHI_w/m2': 65, 'air_temperature_celcius': 7, 'windspeed_knots': 3.1, 'latitude': -37.562, 'producer': 1, 'precipitation_flag': 'I', 'created_date': '25/06/21 23:30:43', 'relative_humidity': 40.5, 'longitude': 148.05}
Message published successfully. {'max_wind_speed': 12.0, 'precipitation_value': 0.0, 'date': '21/03/19', 'GHI_w/m2': 82, 'air_temperature_celcius': 9, 'windspeed_knots': 4.6, 'latitude': -37.444, 'producer': 1, 'precipitation_flag': 'I', 'created_date': '25/06/21 23:30:53', 'relative_humidity': 42.0, 'longitude': 148.101}
Message published successfully. {'max_wind_speed': 12.0, 'precipitation_value': 0.0, 'date': '22/03/19', 'GHI_w/m2': 156, 'air_temperature_celcius': 19, 'windspeed_knots': 6.2, 'latitude': -36.4553, 'producer': 1, 'precipitation_flag': 'I', 'created_date': '25/06/21 23:31:03', 'relative_humidity': 55.3, 'longitude': 142.8786}
Message published successf

KeyboardInterrupt: 